### logistic regression has reached it limit at AUC of 0.72, curious about if it is because of algorithm restriction, so get a small dataset to try one ensemble algorithm, random forest

In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('final_dummies_validation.csv') 

In [2]:
data=data.sample(frac=0.2,replace=False)

In [3]:
data=data.drop([data.columns[0],'review_id','votes_total_user'],1)
data=data.drop([data.columns[0]],1)

In [4]:
data.shape

(78274, 31)

In [11]:
data_train, data_test, Y_train, Y_test = train_test_split(data,data['Label'], test_size=0.2)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
import numpy as np
import scipy as sp

C:\Users\xulei\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
data_train=pd.DataFrame(data_train)
data_test=pd.DataFrame(data_test)

In [13]:
data_train.head()

,text,Label,1.0_Price,2.0_Price,3.0_Price,4.0_Price,normalized_votes,0.0_Waiter Service,1.0_Waiter Service,1_open,...,1.0_Noise Level,0.0_Take-out,1.0_Take-out,0.0_Attire,1.0_Attire,1_Ambience_score,2_Ambience_score,3_Ambience_score,4_Ambience_score,6_Ambience_score
259525,I have gave this place a lot of chances to cha...,1.0,1.0,0.0,0.0,0.0,-0.206797,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
220077,"Man, this place is awesome. It's not just a su...",1.0,0.0,1.0,0.0,0.0,-0.203347,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
220870,I love chinese food and I love mexican food. W...,1.0,1.0,0.0,0.0,0.0,0.097952,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
237175,Yummy! Great service family restaurant...and t...,0.0,1.0,0.0,0.0,0.0,-0.203347,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
322838,Best French fries in town! Awesome burgers! On...,0.0,1.0,0.0,0.0,0.0,-0.207084,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [14]:
vectorizer = CountVectorizer(max_features=None,ngram_range=(1,2)) 
vectorizer.fit(data_train['text'].copy())
X_train_text= vectorizer.transform(data_train['text'].copy())
X_test_text=vectorizer.transform(data_test['text'].copy())
X_train_other=sp.sparse.csr_matrix(data_train.drop(['text','Label'],1).astype(float))
X_train=sp.sparse.hstack([X_train_text,X_train_other]) #This is the variable set, suppose ‘a’ is the sparse matrix from CountVectorizer/balabala fit-transformed data.
X_test_other=sp.sparse.csr_matrix(data_test.drop(['text','Label'],1).astype(float))
X_test=sp.sparse.hstack([X_test_text,X_test_other])

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve,auc

In [20]:
rf_def = RandomForestClassifier(criterion='entropy', n_estimators=100)
rf_def = rf_def.fit(X_train, Y_train)
rf_pred = rf_def.predict_proba(X_test)

In [21]:
def plotROC(X_test_proba,Y_test,label_string):
    fpr, tpr, thresholds = roc_curve(Y_test, X_test_proba)
    roc_auc = auc(fpr, tpr)
    c = (np.random.rand(), np.random.rand(), np.random.rand())
    fig=plt.figure()
    plt.plot(fpr, tpr, color = c, label = label_string + ' (AUC = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC_{0}'.format(label_string))
    plt.legend(loc=4,prop={'size':10})
    plt.savefig('{0}.pdf'.format(label_string))
    return roc_auc

In [24]:
import matplotlib.pyplot as plt

In [25]:
auc_rm=plotROC(rf_pred[:,1],Y_test,'Random Forest')